### 按照官网的命令安装MFA

In [ ]:
!git clone https://github.com/AlexandaJerry/PPASR.git
%cd /content/PPASR

In [18]:
wav_dir = 'inputs'#@param {type:"string"}
wav_path = '/content/PPASR/' + wav_dir
textgrid_dir = 'outputs'#@param {type:"string"}
textgrid_path = '/content/PPASR/' + textgrid_dir
acoustic_model = '/content/PPASR/mandarin.zip'
dictionary = '/content/PPASR/mandarin_pinyin.txt'
%mkdir $wav_dir
%mkdir $textgrid_dir

mkdir: cannot create directory ‘inputs’: File exists
mkdir: cannot create directory ‘outputs’: File exists


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
file_name = 'data.zip'#@param {type:"string"}
file_path = "/content/drive/MyDrive/" + file_name
!unzip -j $file_path "*/*.wav" -d $wav_path
!unzip -j $file_path "*/*.txt" -d $wav_path

### 安装好后查看MFA的版本
#### 安装无误会显示2.0.6

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install_miniconda()
!conda create -n aligner -c conda-forge montreal-forced-aligner

In [27]:
!source activate aligner; \
mfa version

/usr/local/envs/aligner/lib/python3.10/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(
2.0.6




---



## 短音频对齐示范

---


### mfa align命令必须输入的参数包括四个路径：
#### 01音频和转写的路径 02词典的路径 03声学模型的路径 04输出textgrid的路径
#### 我们的词典是拼音词典mandarin_pinyin.txt
#### 我们的声学模型路径是mandarin.zip

---


### mfa align命令可以自由选择输入的参数包括：
#### --clean → 清除上次对齐时保留的缓存文件
#### -s number → 文件名中的前多少个字母为发音人的ID
#### 例如前两位文件名是发音人的ID，我们会用-s 2进行发音人的区分
#### --beam number → 对齐搜索的精度默认数字为10
#### 正常情况不需要修改beam，如果无法对齐则需要增大beam
#### --overwrite → 覆盖上次对齐好的Textgrid

---

### 示范命令：
!mfa align --clean -s 2 wav_txt_dir dictionary_path acoustic_model_dir textgrids_dir --beam 10 --overwrite


In [32]:
# Slicer arguments
speaker_id = 6 #@param {type:"number"}
beam_number = 100 #@param {type:"number"}

!source activate aligner; \
mfa align --clean -s $speaker_id $wav_dir $dictionary $acoustic_model $textgrid_dir --beam $beam_number --overwrite

/usr/local/envs/aligner/lib/python3.10/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(
INFO - Setting up corpus information...
INFO - Loading corpus from source files...
100% 87/87 [00:01<00:00, 81.62it/s]
INFO - Found 4 speakers across 87 files, average number of utterances per speaker: 21.75
INFO - Initializing multiprocessing jobs...
INFO - Creating corpus split for feature generation...
INFO - Generating base features (mfcc)...
INFO - Generating MFCCs...
 92% 80/87 [00:05<00:00, 13.34it/s]
INFO - Calculating CMVN...
INFO - Creating corpus split with features...
INFO - Compiling training graphs...
100% 87/87 [00:01<00:00, 63.32it/s] 
INFO - Performing first-pass alignment...
INFO - Generating

In [33]:
from google.colab import files
zip_name_you_wish = 'textgrid.zip'#@param {type:"string"}
zip_path = '/content/PPASR/' + zip_name_you_wish
!zip -j $zip_path $textgrid_dir/*
files.download(zip_path)

updating: aam0511_slice_0000.TextGrid (deflated 87%)
updating: aam0511_slice_0001.TextGrid (deflated 87%)
updating: aam0511_slice_0002.TextGrid (deflated 86%)
updating: aam0511_slice_0003.TextGrid (deflated 86%)
updating: aam0511_slice_0004.TextGrid (deflated 87%)
updating: aam0511_slice_0005.TextGrid (deflated 87%)
updating: aam0511_slice_0006.TextGrid (deflated 87%)
updating: aam0511_slice_0007.TextGrid (deflated 87%)
updating: aam0511_slice_0008.TextGrid (deflated 87%)
updating: aam0511_slice_0009.TextGrid (deflated 88%)
updating: aam0511_slice_0010.TextGrid (deflated 87%)
updating: aam0511_slice_0011.TextGrid (deflated 88%)
updating: aam0511_slice_0012.TextGrid (deflated 88%)
updating: aam0511_slice_0013.TextGrid (deflated 88%)
updating: aam0511_slice_0014.TextGrid (deflated 85%)
updating: aam0511_slice_0015.TextGrid (deflated 86%)
updating: aam0511_slice_0016.TextGrid (deflated 87%)
updating: aam0511_slice_0017.TextGrid (deflated 88%)
updating: aam0511_slice_0018.TextGrid (deflate

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>